Okay, the plan is we stick with fixture as a numpy array, but we ditch the tournament class stuff and pull in things to match GA

In [1]:
import numpy as np
from numpy import random as r
from itertools import combinations
import json

In [2]:
# Parameters
teams = ["Adelaide Crows", 
         "Brisbane Lions", 
         "Carlton Blues", 
         "Collingwood Magpies",
         "Essendon Bombers", 
         "Fremantle Dockers", 
         "Geelong Cats", 
         "Gold Coast Suns",
         "Greater Western Sydney Giants", 
         "Hawthorn Hawks", 
         "Melbourne Demons",
         "North Melbourne Kangaroos", 
         "Port Adelaide Power",
         "Richmond Tigers",
         "St Kilda Saints",
         "Sydney Swans",
         "West Coast Eagles",
         "Western Bulldogs"]

team_numbers = {team: number for number, team in enumerate(sorted(teams), start=0)}

locations = ['VIC','NSW','SA','WA','QLD']

location_numbers = {location: number for number, location in enumerate(sorted(locations), start=0)}

home_locations = ['SA','QLD','VIC','VIC','VIC','WA','VIC','QLD','NSW','VIC',
                          'VIC','VIC','SA','VIC','VIC','NSW','WA','VIC']

ranking = [10,2,5,1,11,14,12,15,7,16,4,17,3,13,6,8,18,9] # 2023 pre-finals rankings

team_fans = [60,45,93,102,79,61,81,20,33,72,68,50,60,100,58,63,100,55] # 2023 number of members, in 000's

wins = [11,17,13.5,18,11,10,10.5,9,13,7,16,3,17,10.5,13,12.5,3,12]

stadiums = ['MCG','Marvel','GMHBA','Adelaide Oval','Optus','Gabba','HBS','SCG','Giants']

stadium_numbers = {stadium: number for number, stadium in enumerate(sorted(stadiums), start=0)}

stadium_locations = ['VIC','VIC','VIC','SA','WA','QLD','QLD','NSW','NSW']

home_stadiums = [['Adelaide Oval'],['Gabba'],['MCG','Marvel'],['MCG','Marvel'],['MCG','Marvel'],['Optus'],['GMHBA'],
                 ['HBS'],['Giants'],['MCG'],['MCG'],['Marvel'],['Adelaide Oval'],
                 ['MCG'],['Marvel'],['SCG'],['Optus'],['Marvel']]

home_location_stadiums = [[] for i in range(len(teams))]
for i in range(len(teams)):
    for j in range(len(stadiums)):
        if stadium_locations[j] == home_locations[i]:
            home_location_stadiums[i].append(stadiums[j])

stadium_size = [100,53,40,54,60,39,27,47,24] # Stadium sizes in 000's

rivals = [['Port Adelaide Power'],
          ['Gold Coast Suns','Collingwood Magpies'], # From wikipedia
          ['Essendon Bombers','Richmond Tigers','Collingwood Magpies', 'Fremantle Dockers'],
          ['Carlton Blues','Essendon Bombers','Brisbane Lions','Melbourne Demons','Richmond Tigers','Geelong Cats',
           'Hawthorn Hawks', 'West Coast Eagles','Port Adelaide Power'],
          ['Carlton Blues','Collingwood Magpies','Richmond Tigers','Hawthorn Hawks','North Melbourne Kangaroos'],
          ['Carlton Blues','West Coast Eagles'],
          ['Collingwood Magpies','Hawthorn Hawks'],
          ['Brisbane Lions'],
          ['Sydney Swans','Western Bulldogs'],
          ['Hawthorn Hawks','Geelong Cats','Essendon Bombers', 'North Melbourne Kangaroos'],
          ['Collingwood Magpies'],
          ['Essendon Bombers','Hawthorn Hawks'],
          ['Adelaide Crows','Collingwood Magpies'],
          ['Carlton Blues','Collingwood Magpies','Essendon Bombers'],
          ['Sydney Swans'],
          ['West Coast Eagles','St Kilda Saints','Greater Western Sydney Giants','Hawthorn Hawks'],
          ['Fremantle Dockers','Sydney Swans','Collingwood Magpies'],
          ['Greater Western Sydney Giants']]

rivals_num = [[team_numbers[i] for i in rivals[j]] for j in range(len(rivals))]

timeslots = [i for i in range(7)]
timeslot_values = [10,13,5,6,11,5,4] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening']

Ts = range(len(teams))
Ss = range(len(stadiums))
timeslots = range(7)
rounds = range(22)

fixture_matrix = np.array([[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds])


In [3]:
# Daniel's Feasibility Function

def feasibility_print(fixture):
    violated = 0
    critical = 0
    
    for i in Ts: # Each team plays once a week
        for r in rounds:
            if sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for j in Ts for s in Ss for t in timeslots)-1 > 0:
                print(f'Critical Violation: {teams[i]} does not play in round {r}')
            critical += abs(sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for j in Ts for s in Ss for t in timeslots)-1)
            
    
    for i in Ts: # 11 home games 
        #sum(fixture[i, j, s, t, r] for j in self.C for s in self.S for t in self.T for r in self.R) != 11
        if sum(fixture[i, j, s, t, r] for j in Ts for s in Ss for t in timeslots for r in rounds) > 11:
            print(f'Critical Violation: {teams[i]} play more than 11 home games')
        critical += abs(sum(fixture[i,j,stadium_numbers[s],t,r] for j in Ts for s in home_stadiums[i] 
                                 for t in timeslots for r in rounds)-11)
    
    
    for i in Ts: # Cannot play yourself 
        if sum(fixture[i,i,s,t,r] for s in Ss for t in timeslots for r in rounds) > 0:
            print(f'Critical Violation: {teams[i]} play themselves')
        critical += sum(fixture[i,i,s,t,r] for s in Ss for t in timeslots for r in rounds)
        
        for j in Ts: 
            if i != j:
                if max(sum(fixture[i,j,s,t,r] for s in Ss for t in timeslots for r in rounds)-1,0) > 0:
                    print(f'Violation: Teams {teams[i]} & {teams[j]} verse eachother more than twice')
                if max(1-sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for s in Ss for t in timeslots for r in rounds),0)>0:
                    print(f'Violation: Teams {teams[i]} & {teams[j]} do not verse eachother')
                violated += max(sum(fixture[i,j,s,t,r] for s in Ss for t in timeslots for r in rounds)-1,0) #Play more than twice 
                violated += max(1-sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for s in Ss for t in timeslots for r in rounds),0) #Play at least once 
                
                
    for i in Ts: #At least a 5 day break 
        for r in rounds[:-1]:
            if max(0,sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for j in Ts for s in Ss for t in [5,6])+ 
                            sum(fixture[i,j,s,t,r+1]+fixture[j,i,s,t,r+1] for j in Ts for s in Ss for t in [0]) - 1) > 0:
                print(f'Violation: Team {teams[i]} does not get a 5 day break in round{r}')
            violated += max(0,sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for j in Ts for s in Ss for t in [5,6])+ 
                            sum(fixture[i,j,s,t,r+1]+fixture[j,i,s,t,r+1] for j in Ts for s in Ss for t in [0]) - 1)
            
    
    # Three games in a row outside home location
    for i in Ts:
        for r in rounds[:-2]:
            if max(0,1-sum(fixture[j,i,stadium_numbers[s],t,r_]+fixture[i,j,stadium_numbers[s],t,r_] for j in Ts for s in home_location_stadiums[i] 
                                 for t in timeslots for r_ in range(r,r+3))) > 0:
                print(f'Violation: Team {teams[i]} play more than 3 consecutive games in round {r}')
            violated += max(0,1-sum(fixture[j,i,stadium_numbers[s],t,r_]+fixture[i,j,stadium_numbers[s],t,r_] for j in Ts for s in home_location_stadiums[i] 
                                 for t in timeslots for r_ in range(r,r+3)))
       
    # Four away games in a row
    for i in Ts:
        for r in rounds[:-3]:
            if  max(0,1-sum(fixture[i,j,s,t,r_] for j in Ts for s in Ss for t in timeslots for r_ in range(r,r+4)))>1:
                print(f'Violation: Team {teams[i]} plays 4 games in a row in round {r}')
            violated += max(0,1-sum(fixture[i,j,s,t,r_] for j in Ts for s in Ss for t in timeslots for r_ in range(r,r+4)))
    
    
    # Constraint 7: 2+ games in one day in the same stadium
    for r in rounds:
        for s in Ss:
            
            if max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for t in [5, 6])-1) > 0:
                print(f'Violation: Only 1 Sunday game in round {r}, stadium {s}')
            if max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for t in [2, 3, 4])-1)>0:
                print(f'Violation: Only 1 Saturday game in round {r}, stadium {s}')
            
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for t in [5, 6])-1)
            
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for t in [2, 3, 4])-1)
            
            for t in [0,1]:
                if max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts)-1)>0:
                    print(f'Violation: Stadium {s} overbooked in timeslot {timeslot_names[i]}')
                violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts)-1)
    
    
    # Constraint: No more than two games in any timeslot, and only one on Thursday and Friday night, at least one in each
    for r in rounds:
        
        for t in [2,3,4,5,6]:
            if max(0,1-sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss))>0:
                print('Violation: No game in round {r}, timeslot {timeslot_names[i]}')
            if max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)-2) > 0:
                print(f'Violation: More than 2 simultaneous games in round {r}, timeslot {timeslot_names[i]}')
            violated += max(0,1-sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)) # At least one game each timeslot
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)-2)
        
        # Why seperate thurs and fri 
        for t in [0,1]:
            if max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)-1)>0:
                print(f'Violation: More than one game in round {r}, timeslot {timeslot_names[i]}')        
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)-1) # One game 
            
            
    return violated, critical

In [4]:
# Daniel's Feasibility Function

def feasibility(fixture):
    violated = 0
    critical = 0
    
    for i in Ts: # Each team plays once a week
        for r in rounds:
            critical += abs(sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for j in Ts for s in Ss for t in timeslots)-1)
            
    
    for i in Ts:
        critical += abs(sum(fixture[i,j,stadium_numbers[s],t,r] for j in Ts for s in home_stadiums[i] 
                                 for t in timeslots for r in rounds)-11)
    
    
    for i in Ts:
        critical += sum(fixture[i,i,s,t,r] for s in Ss for t in timeslots for r in rounds)
        
        for j in Ts:
            if i != j:
                violated += max(sum(fixture[i,j,s,t,r] for s in Ss for t in timeslots for r in rounds)-1,0)
                violated += max(1-sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for s in Ss for t in timeslots for r in rounds),0)
                
                
    for i in Ts:
        for r in rounds[:-1]:
            violated += max(0,sum(fixture[i,j,s,t,r] + fixture[j,i,s,t,r] for j in Ts for s in Ss for t in [5,6])+ 
                            sum(fixture[i,j,s,t,r+1]+fixture[j,i,s,t,r+1] for j in Ts for s in Ss for t in [0]) - 1)
            
    
    # Three games in a row outside home location
    for i in Ts:
        for r in rounds[:-2]:
            violated += max(0,1-sum(fixture[j,i,stadium_numbers[s],t,r_]+fixture[i,j,stadium_numbers[s],t,r_] for j in Ts for s in home_location_stadiums[i] 
                                 for t in timeslots for r_ in range(r,r+3)))
       
    # Four away games in a row
    for i in Ts:
        for r in rounds[:-3]:
            violated += max(0,1-sum(fixture[i,j,s,t,r_] for j in Ts for s in Ss for t in timeslots for r_ in range(r,r+4)))
    
    
    # Constraint 7: 2+ games in one day in the same stadium
    for r in rounds:
        for s in Ss:
            
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for t in [5, 6])-1)
            
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for t in [2, 3, 4])-1)
            
            for t in [0,1]:
                violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts)-1)
    
    
    # Constraint: No more than two games in any timeslot, and only one on Thursday and Friday night, at least one in each
    for r in rounds:
        
        for t in [2,3,4,5,6]:
            violated += max(0,1-sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)) # At least one game each timeslot
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)-2)
        
        for t in [0,1]:
            #Changed this from 'abs' to 'max'
            violated += max(0,sum(fixture[i,j,s,t,r] for i in Ts for j in Ts for s in Ss)-1) # One game
            
            
    return violated, critical

In [5]:
# Attractiveness Function Parameters, adjust them as needed
alpha = 1.0
beta = 1.0
gamma = 1.0
sigma = 1.0
xi = 1.0

def attractiveness(i, j, s, t, r):
    score = 1
    if r == 0:
        score *= 4
    elif r == 1:
        score *= 2
    elif r == 21:
        score *= 1.5
    
    if j in [team_numbers[i] for i in rivals[0]]:
        score *= 1+alpha
    
    score /= np.sqrt(1+abs(ranking[i]-ranking[j]))
    score  /= np.sqrt(ranking[i]+ranking[j])
    
    if stadium_locations[s] == home_locations[j]:
        score *= (1+beta)
        
    score *= np.sqrt(stadium_size[s])
    score *= (team_fans[i]+0.5*team_fans[j])
    
    score *= timeslot_values[t]
    
    return score

In [6]:
def probability_win(i, j, s):
    probability = wins[i]/(wins[i]+wins[j])
    if stadiums[s] not in home_location_stadiums[j]:
        probability += (1-probability)/2.5
    elif stadiums[s] not in home_stadiums[j]:
        probability += (1-probability)/4
    else:
        probability += (1-probability)/10
        
    return probability

In [7]:
def expected_win_variance(fixture):
    results = []
    expected_wins = [0]*18
    for r in rounds:
        for i in Ts:
            for j in Ts:
                for s in Ss:
                    for t in timeslots:
                        expected_wins[i] += probability_win(i, j, s)*fixture[i,j,s,t,r]
                        expected_wins[i] += (1-probability_win(j, i, s))*fixture[j,i,s,t,r]
        
        results.append(np.var(expected_wins))
    
    # print(results)
    return sum((i+1)*results[i] for i in range(len(results)))

In [8]:
def fixture_attractiveness(fixture,max_value,violated_factor,critical_factor,equality_factor):
    total_score = 0
    
    for r in rounds:
        for t in timeslots:
            value = 0
            for i in Ts:
                for j in Ts:
                    for s in Ss:
                        value += attractiveness(i, j, s, t, r)*fixture[i,j,s,t,r]
            
            total_score += min(max_value,value)

    violated, critical = feasibility(fixture)
    equality = equality_factor*expected_win_variance(fixture)

    # print(total_score)
    # print('-', violated_factor*violated)
    # print('-', critical_factor*critical)
    # print('-', equality)
    
    return total_score - violated_factor*violated - critical_factor*critical - equality


In [9]:
def objective_value(fixture):

    violated, critical = feasibility(fixture)

    violated_penalty = 2*10^4
    critical_penalty = 10^6
    # for i in Ts:
    #     for j in Ts:
    #         for s in Ss:
    #             for r in rounds:
    #                 for t in timeslots:
    #                     print(attractiveness(i,j,s,t,r))

    objective_value = - violated_penalty*violated - critical_penalty*critical + fixture_attractiveness(fixture, 2*10^4, critical_penalty, violated_penalty, 10^3) 

    return objective_value

In [36]:
def find_stadium(fixture, i, t, r):
    # finds a stadium for a home team in round r timeslot t
    for stad in home_stadiums[i]:
        if np.sum(fixture[:,:,stadium_numbers[stad],t,r]) == 0:
            return stadium_numbers[stad]

    # can't find a stadium then find a stadium that is at the very least in the same state
    for stad in home_location_stadiums[i]:
        if np.sum(fixture[:,:,stadium_numbers[stad],t,r]) == 0:
            return stadium_numbers[stad]
        
    # no stadium can be found
    return -1

In [18]:
def simulated_annealing_solve(initial_schedule, cooling_type, starting_temp, final_temp, cooling_size):
    #Establish cooling schedule
    if cooling_type == 'geometric':
        #Geometric cooling for lazy - Set an initial T, a geometric factor, and a size
        initial_T = starting_temp
        decay = (final_temp/starting_temp)**(1/cooling_size)
        cooling_schedule = [initial_T*decay**i for i in range(0,cooling_size)]
        #print(cooling_schedule)

    current_schedule = np.copy(initial_schedule)
    current_objective = objective_value(current_schedule)
    best_schedule = np.copy(current_schedule)
    best_objective = current_objective

    #NB the way of doing simulated annealing in the lecture notes is a bit off, we just loop on T, rather than looping on T and nested loop of certain number iterations at each T
    for T in cooling_schedule:
        #Currently very inefficient - calling the whole objective function each time, rather than having random_neighbourhood return a delta objective value
        new_schedule = random_neighbour(current_schedule)
        new_objective = objective_value(new_schedule)
        #print(np.exp(-(new_objective-current_objective)/T))
        if new_objective > current_objective or r.random() <= np.exp((new_objective-current_objective)/T):
            current_schedule = new_schedule
            current_objective = new_objective
            print(int(current_objective))
            
            if current_objective > best_objective:
                best_objective = current_objective
                best_schedule = np.copy(current_schedule)

    return best_schedule, best_objective

In [24]:
#A schedule is  a 5d array, where S[i,j,s,t,r] is a boolean that indicates whether team i plays team j in stadium k at timeslot l in round m

def random_neighbour(schedule):
    #Function that gets a random neighbour, choosing which neighbourhood to explore with a particular probability
    #Parameters for tuning likeliness of using a particular neighbourhood function
    a = 0.33
    b = 0.66
    p = r.rand()
    if p<a:
        print('Swapping a home and away')
        new_schedule = random_neighbour_home_swap(schedule)
    elif p>=a and p<b:
        print('Moving a match')
        new_schedule = random_neighbour_match_move(schedule)
    else:
        print('Swapping a double-play')
        new_schedule = random_neighbour_double_swap(schedule)
    
    return new_schedule
    
def random_neighbour_home_swap(schedule):
    #Function that swaps a random match from home to away
    new_schedule = schedule.copy()
    
    #Pick two teams
    i, j = r.choice(range(0,18), 2, replace=False)

    #Choose one of the one plus games they play
    #Aight, so this returns a list of indices, each should be (i,j, non ij index of any actual ij matches)
    i_home = [(i,j) + tuple(index) for index in zip(*np.nonzero(schedule[i,j,:,:,:]))]
    j_home = [(j,i) + tuple(index) for index in zip(*np.nonzero(schedule[j,i,:,:,:]))]
    
    #Concatenate the two lists of indices for i vs j matches
    all_matches = i_home+j_home
    #print(all_matches)
    
    #Pick one
    if len(all_matches) <= 0:
        #Teams don't play - a constraint violation, but can occur. In this case, return the schedule unchanged
        return new_schedule
    old_match_index = r.randint(len(all_matches))
    old_match = all_matches[old_match_index]
    
    #Take the old match off the schedule
    new_schedule[old_match] = 0
    
    #Flip who plays at home
    new_match = list(old_match)
    new_match[0], new_match[1] = new_match[1], new_match[0]
    
    #Now find home stadium
    s = find_stadium(new_schedule, new_match[0], new_match[3], new_match[4])
    if s == -1:
        #Couldn't find home stadium, make no change
        new_schedule[old_match] = 1
        return new_schedule
    else:
        new_match[2] = s
        new_match = tuple(new_match)
        new_schedule[new_match] = 1

        #Return our new schedule
        return new_schedule
    
def random_neighbour_match_move(schedule):
    #Function that moves a random match to a different time
    new_schedule = schedule.copy()
    
    #Pick hometeam
    i = r.randint(18)
    #Gets all the homegames the team plays
    homegames = [[i] + list(index) for index in zip(*np.nonzero(schedule[i,:,:,:,:]))]
    #Pick one
    old_match = homegames[r.randint(len(homegames))]
    
    
    done = False
    while done == False:
        #Pick a new timeslot and round
        t = r.randint(7)
        round = r.randint(22)
        
        #If noone else is playing in this stadium at this time, we move the match here
        if np.sum(schedule[:,:,old_match[2],t,round]) == 0:
            done = True
            new_schedule[old_match[0], old_match[1], old_match[2], t, round] = 1
            new_schedule[tuple(old_match)] = 0
    #Return the new schedule
    return new_schedule
    
    
    
def random_neighbour_double_swap(schedule):
    #Function that takes two random double matches and swaps two of the teams between matches
    new_schedule = schedule.copy()
    #Find two pairs of pairs of teams that play twice
    done = False
    while done == False:
        #pick two teams
        i, j, k, l = r.choice(range(0,18), 4, replace=False)
        if np.sum(schedule[i,j,:,:,:])+np.sum(schedule[j,i,:,:,:]) == 2 and np.sum(schedule[k,l,:,:,:]) + np.sum(schedule[l,k,:,:,:])  == 2:
            done = True
    
    #Find all the matches i  and j play together
    i_home = [(i,j) + tuple(index) for index in zip(*np.nonzero(schedule[i,j,:,:,:]))]
    j_home = [(j,i) + tuple(index) for index in zip(*np.nonzero(schedule[j,i,:,:,:]))]
    i_j_matches = i_home + j_home
    
    #Find all the matches k and l play together
    k_home = [(k,l) + tuple(index) for index in zip(*np.nonzero(schedule[k,l,:,:,:]))]
    l_home = [(l,k) + tuple(index) for index in zip(*np.nonzero(schedule[l,k,:,:,:]))]
    k_l_matches = k_home + l_home
    
    #Picks one of the i vs j matches, and one of the k vs l matches
    i_j_to_swap = i_j_matches[r.randint(len(i_j_matches))]
    k_l_to_swap = k_l_matches[r.randint(len(k_l_matches))]
    
    #Gets indices of swapped matches - We're swapping the away teams of the two matches
    i_l_match = (i_j_to_swap[0], l) + i_j_to_swap[2:]
    k_j_match = (k_l_to_swap[0], j) + k_l_to_swap[2:]

    
    #Performs the swap
    new_schedule[i_j_to_swap] = 0
    new_schedule[k_l_to_swap] = 0
    new_schedule[i_l_match] = 1
    new_schedule[k_j_match] = 1
    
    #Return the new schedule
    return new_schedule

In [21]:
initial_fixture = np.transpose(np.load('solutions/mip_initial_fixture.npy'),(4, 3, 2, 1, 0))
print(np.shape(initial_fixture))

(18, 18, 9, 7, 22)


In [39]:
#simulated_annealing_solve(initial_schedule, cooling_type, starting_temp, final_temp, cooling_size)
print(objective_value(initial_fixture))
final_fixture, final_result = simulated_annealing_solve(initial_fixture, 'geometric', 10**3, 20, 100)
print(objective_value(initial_fixture))
print(objective_value(final_fixture))

-1595.1940902246556
Swapping a home and away
-1890
Swapping a home and away
-1963
Swapping a double-play
-2016
Swapping a home and away
-2049
Swapping a home and away
-1995
Swapping a home and away
Swapping a double-play
-2190
Moving a match
-2460
Swapping a home and away
-2611
Swapping a double-play
Swapping a double-play
-2796
Moving a match
-2923
Swapping a double-play
-3080
Swapping a double-play
Moving a match
-3299
Moving a match
-3327
Swapping a double-play
-3459
Swapping a double-play
-3706
Moving a match
Moving a match
-3777
Swapping a double-play
-3744
Swapping a double-play
-3719
Swapping a double-play
Moving a match
Swapping a double-play
-3879
Moving a match
-3923
Swapping a home and away
-3885
Moving a match
-4024
Swapping a double-play
Moving a match
Moving a match
Moving a match
-4421
Swapping a double-play
-4616
Swapping a home and away
-4690
Moving a match
-4725
Swapping a home and away
-4725
Swapping a home and away
-4786
Moving a match
-4899
Swapping a home and away

KeyboardInterrupt: 

In [60]:
print('Initial with penalties', tourn.fixture_attractiveness(initial_fixture, 10**7, 10**6, 10**4))
print('Initial without penalties', tourn.fixture_attractiveness(initial_fixture, 10**7))
print('Final with penalties: ', tourn.fixture_attractiveness(final_fixture, 10**7, 10**6, 10**4))
print('Final without penalties: ', tourn.fixture_attractiveness(final_fixture,10**7))
print('Final infeasibilities:', tourn.feasibility(final_fixture) )

Initial with penalties -109908081.18967468
Initial without penalties 16091918.810325325
Final with penalties:  -103734830.15791512
Final without penalties:  20265169.842084877
Final infeasibilities: (124.0, 0.0)


In [65]:
print(np.shape(tourn.fixture_matrix))

(18, 18, 9, 9, 22)


In [59]:
np.save('simulated_annealing_fixture.npy', final_fixture)

In [28]:
#Scrap code testing
a = 4
b = a
a = 3
print(a, b)
print(stadium_numbers['MCG'])

3 4
5
